<a href="https://colab.research.google.com/github/scfaundez/Macro_Finances/blob/main/nowcasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#Load data (load_data.py)
#-------------------------------------------------Libraries
import pandas as pd
import numpy as np
import os

#-------------------------------------------------Read data functions
def load_data(datafile,Spec,sample = None):

    # loadData Load vintage of data from file and format as structure
    #
    # Description:
    #
    #   Load data from file
    #
    # Input Arguments:
    #
    #   datafile - filename of Microsoft Excel workbook file
    #
    # Output Arguments:
    #
    # Data - structure with the following fields:
    #   .    X : T x N numeric array, transformed dataset
    #   . Time : T x 1 numeric array, date number with observation dates
    #   .    Z : T x N numeric array, raw (untransformed) dataset

    """
    Python Version Notes:
        The original matlab function can load raw data from MATLAB formatted binary (.mat) file.
        However, in the Python version I have removed this feature.

        In transformData(), the formula_dict dictionary contains functions that transform the data. I could have done
        it in a similar fashion as the Matlab code, however I find it easier to read the code when going through the if and elif statements

        When converting dates to ordinal format, we need to add 366 days to match with Matlab date numeric values
    """

    if not os.path.splitext(datafile)[1] in [".xlsx",".xls"]:
        ValueError("File is not an EXCEL FILE")

    Z,Time,Mnem = readData(datafile)
    #    Z : raw (untransformed) observed data
    # Time : observation periods for the time series data
    # Mnem : series ID for each variable

    # Sort data based on model specification
    Z,_ = sortData(Z.copy(),Mnem.copy(),Spec)

    # since now Mnem == Spec.SeriesID
    del Mnem

    # Transform data based on model specification
    X,Time,Z = transformData(Z.copy(),Time.copy(),Spec)

    # Drop data not in estimation sample
    if sample != None:
        X,Time,Z = dropData(X.copy(),Time.copy(),Z.copy(),sample)

    return X,Time,Z


def readData(datafile):

    # readData Read data from Microsoft Excel workbook file

    dat  = pd.read_excel(datafile)
    Mnem = np.array([i for i in list(dat.columns) if i != "Date"])
    Z    = dat[Mnem].to_numpy(copy=True)
    Time = dat.Date.apply(lambda x: x.toordinal()+366).to_numpy(copy= True)

    return Z,Time,Mnem

def sortData(Z,Mnem,Spec):

    # sortData Sort series by order of model specification

    # Drop series not in Spec
    inSpec = np.in1d(Mnem,Spec.SeriesID)
    Mnem   = Mnem[inSpec]
    Z      = Z[:,inSpec]

    # Sort series by ordering of Spec
    permutation = np.array([np.where(Mnem == i)[0][0] for i in Spec.SeriesID])
    Mnem        = Mnem[permutation]
    Z           = Z[:,permutation]

    return Z, Mnem

def transformData(Z,Time,Spec):

    # transformData Transforms each data series based on Spec.Transformation
    #
    # Input Arguments:
    #
    #      Z : T x N numeric array, raw (untransformed) observed data
    #   Spec : structure          , model specification
    #
    # Output Arguments:
    #
    #      X : T x N numeric array, transformed data (stationary to enter DFM)

    """
    Transformation notes:
        'lin' = Levels (No Transformation)
        'chg' = Change (Difference)
        'ch1' = Year over Year Change (Difference)
        'pch' = Percent Change
        'pc1' = Year over Year Percent Change
        'pca' = Percent Change (Annual Rate)
        'log' = Natural Log
    """

    T,N          = Z.shape
    X            = np.empty((T, N))
    X[:]         = np.nan
    Freq_dict    = {"m":1,"q":3}
    formula_dict = {"lin":lambda x:x*2,
                    "chg":lambda x:np.append(np.nan,x[t1+step::step] - x[t1:-1-t1:step]),
                    "ch1":lambda x:x[12+t1::step] - x[t1:-12:step],
                    "pch":lambda x:(np.append(np.nan,x[t1+step::step]/x[t1:-1-t1:step]) - 1)*100,
                    "pc1":lambda x:((x[12+t1::step]/x[t1:-12:step])-1)*100,
                    "pca":lambda x:(np.append(np.nan,x[t1+step::step]/x[t1:-step:step])**(1/n) - 1)*100,
                    "log":lambda x:np.log(x)
    }

    for i in range(N):
        formula = Spec.Transformation[i]
        freq    = Spec.Frequency[i]
        step    = Freq_dict[freq] # time step for different frequencies based on monthly time
        t1      = step -1         # assume monthly observations start at beginning of quarter (subtracted 1 for indexing)
        n       = step/12         # number of years, needed to compute annual % changes
        series  = Spec.SeriesName[i]

        if formula == 'lin':
            X[:,i] = Z[:,i].copy()
        elif formula == 'chg':
            X[t1::step,i] = formula_dict['chg'](Z[:,i].copy())
        elif formula == 'ch1':
            X[12+t1::step, i] = formula_dict['ch1'](Z[:, i].copy())
        elif formula == 'pch':
            X[t1::step, i] = formula_dict['pch'](Z[:, i].copy())
        elif formula == 'pc1':
            X[12+t1::step, i] = formula_dict['pc1'](Z[:, i].copy())
        elif formula == 'pca':
            X[t1::step, i] = formula_dict['pca'](Z[:, i].copy())
        elif formula == 'log':
            X[:, i] = formula_dict['log'](Z[:, i].copy())
        else:
            ValueError("{}: Transformation is unknown".format(formula))

    # Drop first quarter of observations
    # since transformations cause missing values

    return X[3:,:],Time[3:],Z[3:,:]


def dropData(X,Time,Z,sample):

    # dropData Remove data not in estimation sample

    filter_index = Time >= sample
    X            = X[filter_index,:].copy()
    Time         = Time[filter_index].copy()
    Z            = Z[filter_index, :].copy()

    return X,Time,Z

In [ ]:
# Load model specification for a dynamic factor model (load_spec.py)
#-------------------------------------------------Libraries
import pandas as pd
import numpy as np
import re


#-------------------------------------------------load_spec class
class load_spec():

    # loadSpec Load model specification for a dynamic factor model (DFM)
    #
    # Description:
    #
    #   Load model specification  'Spec' from a Microsoft Excel workbook file
    #   given by 'filename'.
    #
    # Input Arguments:
    #
    #   filename -
    #
    # Output Arguments:
    #
    # spec - 1 x 1 structure with the following fields:
    #     . series_id
    #     . name
    #     . frequency
    #     . units
    #     . transformation
    #     . category
    #     . blocks
    #     . BlockNames

    """
    Python Version Notes:

    spec is a dictionary containing the fields:
        . series_id
        . name
        . frequency
        . units
        . transformation
        . category
        . blocks
        . BlockNames
    """


    def __init__(self,filename):

        # Find and drop series from Spec that are not in Model
        raw         = pd.read_excel(filename)
        raw.columns = [i.replace(" ","") for i in  raw.columns]
        raw         = raw[raw["Model"] == 1].reset_index(drop = True)

        # Sort all fields of 'Spec' in order of decreasing frequency
        frequency    = ['d','w','m','q','sa','a']
        permutations = []
        for freq in frequency:
            permutations+= list(raw[raw.Frequency == freq].index)
        raw = raw.loc[permutations,:]

        # Parse fields given by column names in Excel worksheet
        fldnms = ['SeriesID','SeriesName','Frequency','Units','Transformation','Category']
        for field in fldnms:
            if field in raw.columns:
                setattr(self,field,raw[field].to_numpy(copy=True))
            else:
                raise ValueError("{} raise ValueError(column missing from model specification.".format(field))

        # Parse blocks
        jColBlock             = list(raw.columns[raw.columns.str.contains("Block", case = False)])
        Blocks                = raw[jColBlock].copy()
        Blocks[Blocks.isna()] = 0

        if not (Blocks.iloc[:,0] == 1).all():
            raise ValueError("All variables must load on global block.")
        else:
            self.Blocks = Blocks.to_numpy(copy=True)
        self.BlockNames = [re.sub("Block[0-9]+-","",i) for i in jColBlock]

        # Transformations
        transformation = {'lin':'Levels (No Transformation)',
                          'chg':'Change (Difference)',
                          'ch1':'Year over Year Change (Difference)',
                          'pch':'Percent Change',
                          'pc1':'Year over Year Percent Change',
                          'pca':'Percent Change (Annual Rate)',
                          'cch':'Continuously Compounded Rate of Change',
                          'cca':'Continuously Compounded Annual Rate of Change',
                          'log':'Natural Log'}

        self.UnitsTransformed = np.array([transformation[i] for i in self.Transformation])

        # Summarize model specification
        print('\n Table 1: Model specification \n')
        print(pd.DataFrame({"SeriesID"         :self.SeriesID,
                            "SeriesName"       :self.SeriesName,
                            "Units"            :self.Units,
                            "UnitsTransformed" :self.UnitsTransformed}))

In [ ]:
#-------------------------------------------------Libraries
import numpy as np
from scipy.signal import lfilter
from scipy.interpolate import CubicSpline


#-------------------------------------------------remNaNs_spline
def remNaNs_spline(X,options):
    ###  Replication files for:
    ###  ""Nowcasting", 2010, (by Marta Banbura, Domenico Giannone and Lucrezia Reichlin),
    ### in Michael P. Clements and David F. Hendry, editors, Oxford Handbook on Economic Forecasting.
    ###
    ### The software can be freely used in applications.
    ### Users are kindly requested to add acknowledgements to published work and
    ### to cite the above reference in any resulting publications
    #
    #Description:
    #
    #remNaNs    Treats NaNs in dataset for use in DFM.
    #
    #  Syntax:
    #    [X,indNaN] = remNaNs(X,options)
    #
    #  Description:
    #    remNaNs() processes NaNs in a data matrix X according to 5 cases (see
    #    below for details). These are useful for running functions in the
    #    'DFM.m' file that do not take missing value inputs.
    #
    #  Input parameters:
    #    X (T x n): Input data where T gives time and n gives the series.
    #    options: A structure with two elements:
    #      options.method (numeric):
    #      - 1: Replaces all missing values using filter().
    #      - 2: Replaces missing values after removing trailing and leading
    #           zeros (a row is 'missing' if >80# is NaN)
    #      - 3: Only removes rows with leading and closing zeros
    #      - 4: Replaces missing values after removing trailing and leading
    #           zeros (a row is 'missing' if all are NaN)
    #      - 5: Replaces missing values with spline() then runs filter().
    #
    #      options.k (numeric): remNaNs() relies on MATLAB's filter function
    #      for the 1-D filter. k controls the rational transfer function
    #      argument's numerator (the denominator is set to 1). More
    #      specifically, the numerator takes the form 'ones(2*k+1,1)/(2*k+1)'
    #      For additional help, see MATLAB's documentation for filter().
    #
    #  Output parameters:
    #    X: Outputted data.
    #    indNaN: A matrix indicating the location for missing values (1 for NaN).

    T,N    = X.shape
    k      = options["k"]
    indNaN = np.isnan(X)

    if options["method"] == 1:  # replace all the missing values
        for i in range(N): # Loop through columns
            x              = X[:,i].copy()
            x[indNaN[:,i]] = np.nanmedian(x)
            x_MA           = lfilter(np.ones((2*k+1))/(2*k+1),1,np.append(np.append(x[0]*np.ones((k,1)),x),x[-1]*np.ones((k,1))))
            x_MA           = x_MA[(2*k+1) -1:] # Match dimensions
            # replace all the missing values
            x[indNaN[:,i]] = x_MA[indNaN[:,i]]
            X[:,i]         = x # Replace vector

    elif options["method"] == 2: # replace missing values after removing leading and closing zeros
        # Returns row sum for NaN values. Marks true for rows with more than 80% NaN
        rem1    = np.nansum(indNaN, axis =1) > (N * 0.8)
        nanLead = np.cumsum(rem1) == np.arange(1,(T+1))
        nanEnd  = np.cumsum(rem1) == np.arange(T,0,-1)
        nanLE   = nanLead|nanEnd

        # Subsets X
        X      = X[~nanLE,:]
        indNaN = np.isnan(X) # Index for missing values

        for i in range(N): # Loop for each series
            x          = X[:,i].copy()
            isnanx     = np.isnan(x)
            t1         = np.min(np.where(~isnanx)) # First non-NaN entry
            t2         = np.max(np.where(~isnanx)) # Last non-NaN entry

            # Interpolates without NaN entries in beginning and end
            x[t1:t2+1] = CubicSpline(np.where(~isnanx)[0],x[~isnanx])(np.arange(t1,t2+1))
            isnanx     = np.isnan(x)

            # replace NaN observations with median
            x[isnanx]  = np.nanmedian(x)

            # Apply filter
            x_MA       = lfilter(np.ones((2*k+1))/(2*k+1),1,np.append(np.append(x[0]*np.ones((k,1)),x),x[-1]*np.ones((k,1))))
            x_MA       = x_MA[(2*k+1) -1:]

            # Replace nanx wih filtered observations
            x[isnanx]  = x_MA[isnanx]
            X[:,i]     = x

    elif options["method"] == 3:
        rem1    = np.sum(indNaN, axis = 1) == N
        nanLead = np.cumsum(rem1) == np.arange(1,(T+1))
        nanEnd  = np.cumsum(rem1) == np.arange(T,0,-1)
        nanLE   = nanLead|nanEnd

        X      = X[~nanLE,:]
        indNaN = np.isnan(X)

    elif options["method"] == 4: # remove rows with leading and closing zeros & replace missing values
        rem1    = np.sum(indNaN, axis = 1) == N
        nanLead = np.cumsum(rem1) == np.arange(1,(T+1))
        nanEnd  = np.cumsum(rem1) == np.arange(T,0,-1)
        nanLE   = nanLead|nanEnd

        X      = X[~nanLE,:]
        indNaN = np.isnan(X)

        for i in range(N):
            x            = X[:, i].copy()
            isnanx       = np.isnan(x)
            t1           = np.min(np.where(~isnanx))
            t2           = np.max(np.where(~isnanx))
            x[t1:t2 + 1] = CubicSpline(np.where(~isnanx)[0],x[~isnanx])(np.arange(t1,t2+1))
            isnanx       = np.isnan(x)
            x[isnanx]    = np.nanmedian(x)
            x_MA         = lfilter(np.ones((2 * k + 1)) / (2 * k + 1), 1,
                                   np.append(np.append(x[0] * np.ones((k, 1)), x), x[-1] * np.ones((k, 1))))
            x_MA      = x_MA[(2 * k + 1) - 1:]
            x[isnanx] = x_MA[isnanx]
            X[:, i]   = x

    elif options["method"] == 5: # replace missing values
        indNaN = np.isnan(X)

        for i in range(N):
            x            = X[:, i].copy()
            isnanx       = np.isnan(x)
            t1           = np.min(np.where(~isnanx))
            t2           = np.max(np.where(~isnanx))
            x[t1:t2 + 1] = CubicSpline(np.where(~isnanx)[0],x[~isnanx])(np.arange(t1,t2+1))
            isnanx       = np.isnan(x)
            x[isnanx]    = np.nanmedian(x)
            x_MA         = lfilter(np.ones((2 * k + 1)) / (2 * k + 1), 1,
                                   np.append(np.append(x[0] * np.ones((k, 1)), x), x[-1] * np.ones((k, 1))))
            x_MA         = x_MA[(2 * k + 1) - 1:]
            x[isnanx]    = x_MA[isnanx]
            X[:, i]      = x

    return X,indNaN

In [ ]:
#-------------------------------------------------Libraries
import numpy as np
import pandas as pd
#from remNaNs_spline import remNaNs_spline
from scipy.linalg import eig
from scipy.linalg import block_diag


#-------------------------------------------------Dynamic Factor Modeling functions
def dfm(X,Spec,threshold = 1e-5,max_iter = 5000):
    # DFM()    Runs the dynamic factor model
    #
    #  Syntax:
    #    Res = DFM(X,Par)
    #
    #  Description:
    #   DFM() inputs the organized and transformed data X and parameter structure Par.
    #   Then, the function outputs dynamic factor model structure Res and data
    #   summary statistics (mean and standard deviation).
    #
    #  Input arguments:
    #    X: Kalman-smoothed data where missing values are replaced by their expectation
    #    Par: A structure containing the following parameters:
    #      Par.blocks: Block loadings.
    #      Par.nQ: Number of quarterly series
    #      Par.p: Number of lags in transition matrix
    #      Par.r: Number of common factors for each block
    #
    # Output Arguments:
    #
    #   Res - structure of model results with the following fields
    #       . X_sm | Kalman-smoothed data where missing values are replaced by their expectation
    #       . Z | Smoothed states. Rows give time, and columns are organized according to Res.C.
    #       . C | Observation matrix. The rows correspond
    #          to each series, and the columns are organized as shown below:
    #         - 1-20: These columns give the factor loa dings. For example, 1-5
    #              give loadings for the first block and are organized in
    #              reverse-chronological order (f^G_t, f^G_t-1, f^G_t-2, f^G_t-3,
    #              f^G_t-4). Columns 6-10, 11-15, and 16-20 give loadings for
    #              the second, third, and fourth blocks respectively.
    #       .R: Covariance for observation matrix residuals
    #       .A: Transition matrix. This is a square matrix that follows the
    #      same organization scheme as Res.C's columns. Identity matrices are
    #      used to account for matching terms on the left and righthand side.
    #      For example, we place an I4 matrix to account for matching
    #      (f_t-1; f_t-2; f_t-3; f_t-4) terms.
    #       .Q: Covariance for transition equation residuals.
    #       .Mx: Series mean
    #       .Wx: Series standard deviation
    #       .Z_0: Initial value of state
    #       .V_0: Initial value of covariance matrix
    #       .r: Number of common factors for each block
    #       .p: Number of lags in transition equation
    #
    # References:
    #
    #   Marta Banbura, Domenico Giannone and Lucrezia Reichlin
    #   Nowcasting (2010)
    #   Michael P. Clements and David F. Hendry, editors,
    #   Oxford Handbook on Economic Forecasting.

    ## Store model parameters ------------------------------------------------

    # DFM input specifications: See documentation for details
    Par = {}
    Par["blocks"] = Spec.Blocks.copy()                                  # Block loading structure
    Par["nQ"]     = (Spec.Frequency == "q").sum()                       # Number of quarterly series
    Par["p"]      = 1                                                   # Number of lags in autoregressive of factor (same for all factors)
    Par["r"]      = np.ones((1,Spec.Blocks.shape[1])).astype(np.int64)  # Number of common factors for each block
    # Par.r(1) =2;
    # Display blocks

    print("\n\n\n")
    print("Table 3: Block Loading Structure")
    print(pd.DataFrame(data=Spec.Blocks, index=Spec.SeriesName, columns=Spec.BlockNames))
    print("\n")
    print("Estimating the dynamic factor model (DFM) \n\n")

    T, N   = X.shape
    r      = Par["r"].copy()
    p      = Par["p"]
    nQ     = Par["nQ"]
    blocks = Par["blocks"].copy()

    i_idio = np.append(np.ones(N-nQ),np.zeros(nQ)).reshape((-1,1),order="F") == 1

    # R*Lambda = q; Contraints on the loadings of the quartrly variables
    R_mat = np.array([2,-1,0,0,0,3,0,-1,0,0,2,0,0,-1,0,1,0,0,0,-1]).reshape((4,5))
    q     = np.zeros((4,1))

    # Prepare data -----------------------------------------------------------
    Mx   = np.nanmean(X,axis = 0)
    Wx   = np.nanstd(X,axis = 0,ddof = 1)
    xNaN = (X - np.tile(Mx,(T,1))) / np.tile(Wx,(T,1))

    # Initial Conditions------------------------------------------------------
    optNaN           = {}
    optNaN["method"] = 2 # Remove leading and closing zeros
    optNaN["k"]      = 3 # Setting for filter(): See remNaN_spline

    A,C,Q,R,Z_0,V_0  = InitCond(xNaN.copy(), r.copy(), p, blocks.copy(), optNaN, R_mat.copy(), q, nQ, i_idio.copy())

    # initialize EM loop values
    previous_loglik = -np.inf
    num_iter        = 0
    LL              = [-np.inf]
    converged       = 0

    # y for the estimation is with missing data
    y                = xNaN.copy().T

    # EM LOOP ----------------------------------------------------------------

    # The model can be written as
    # y = C*Z + e;
    # Z = A*Z(-1) + v
    # where y is NxT, Z is (pr)xT, etc

    # Remove the leading and ending nans
    optNaN["method"] = 3
    y_est,_          = remNaNs_spline(xNaN.copy(),optNaN)
    y_est            = y_est.T

    max_iter = 5000
    while num_iter < max_iter and not converged: # Loop until converges or max iter.

        # Applying EM algorithm
        C_new, R_new, A_new, Q_new, Z_0, V_0, loglik = EMstep(y_est, A, C, Q, R, Z_0, V_0, r,p,R_mat,q,nQ,i_idio,blocks)

        C = C_new.copy()
        R = R_new.copy()
        A = A_new.copy()
        Q = Q_new.copy()

        if num_iter > 2: # Check convergence
            converged, decrease = em_converged(loglik,previous_loglik,threshold,1)

        if (num_iter % 10) == 0 and num_iter > 0:
            print("Now running the {}th iteration of max {}".format(num_iter,max_iter))
            print('Loglik: {} (% Change: {})'.format(loglik, 100*((loglik-previous_loglik)/previous_loglik)))

        LL.append(loglik)
        previous_loglik = loglik
        num_iter        += 1

    if num_iter < max_iter:
        print('Successful: Convergence at {} interations'.format(num_iter))
    else:
        print('Stopped because maximum iterations reached')

    # Final run of the Kalman filter
    Zsmooth,_,_,_  = runKF(y,A,C,Q,R,Z_0,V_0)
    Zsmooth        = Zsmooth.T
    x_sm           = np.matmul(Zsmooth[1:,:],C.T) # Get smoothed X

    # Loading the structure with the results --------------------------------
    Res = { "x_sm"     : x_sm.copy(),
            "X_sm" : np.tile(Wx,(T,1)) * x_sm + np.tile(Mx,(T,1)),
            "Z"        : Zsmooth[1:,:].copy(),
            "C"        : C.copy(),
            "R"        : R.copy(),
            "A"        : A.copy(),
            "Q"        : Q.copy(),
            "Mx"       : Mx.copy(),
            "Wx"       : Wx.copy(),
            "Z_0"      : Z_0.copy(),
            "V_0"      : V_0.copy(),
            "r"        : r,
            "p"        : p,
            "loglik"   : LL
    }

    # Display output
    # Table with names and factor loadings
    nQ       = Par["nQ"]
    nM       = Spec.SeriesID.shape[0] - nQ
    nLags    = max(Par["p"],5)
    nFactors = np.sum(Par["r"])

    print("\n Table 4: Factor Loadings for Monthly Series")
    print(pd.DataFrame(Res["C"][:nM,np.arange(0,nFactors*5,5)],
                       columns = Spec.BlockNames,
                       index   = Spec.SeriesName[:nM]))

    print("\n Table 5: Quarterly Loadings Sample (Global Factor)")
    print(pd.DataFrame(Res["C"][(-1-nQ+1):,:5],
                       columns = ['f1_lag0', 'f1_lag1', 'f1_lag2', 'f1_lag3', 'f1_lag4'],
                       index   = Spec.SeriesName[-1-nQ+1:]))

    # Table with AR model on factors (factors with AR parameter and variance of residuals)
    A_terms = np.diag(Res["A"]).copy()
    Q_terms = np.diag(Res["Q"]).copy()

    print('\n Table 6: Autoregressive Coefficients on Factors')
    print(pd.DataFrame({'AR_Coefficient'    : A_terms[np.arange(0,nFactors*5,5)].copy(),
                        'Variance_Residual' : Q_terms[np.arange(0,nFactors*5,5)].copy()},
                       index   = Spec.BlockNames))

    # Table with AR model idiosyncratic errors (factors with AR parameter and variance of residuals)
    print('\n Table 7: Autoregressive Coefficients on Idiosyncratic Component')
    A_len = A.shape[0]
    Q_len = Q.shape[0]

    A_index = np.hstack([np.arange(nFactors*5,nFactors*5+nM),np.arange(nFactors*5+nM,A_len,5)])
    Q_index = np.hstack([np.arange(nFactors*5,nFactors*5+nM),np.arange(nFactors*5+nM,Q_len,5)])

    print(pd.DataFrame({'AR_Coefficient'    : A_terms[A_index].copy(),
                        'Variance_Residual' : Q_terms[Q_index].copy()},
                       index   = Spec.SeriesName))

    return Res

def InitCond(x,r,p,blocks,optNaN,Rcon,q,nQ,i_idio):
    #InitCond()      Calculates initial conditions for parameter estimation
    #
    #  Description:
    #    Given standardized data and model information, InitCond() creates
    #    initial parameter estimates. These are intial inputs in the EM
    #    algorithm, which re-estimates these parameters using Kalman filtering
    #    techniques.
    #
    #Inputs:
    #  - x:      Standardized data
    #  - r:      Number of common factors for each block
    #  - p:      Number of lags in transition equation
    #  - blocks: Gives series loadings
    #  - optNaN: Option for missing values in spline. See remNaNs_spline() for details.
    #  - Rcon:   Incorporates estimation for quarterly series (i.e. "tent structure")
    #  - q:      Constraints on loadings for quarterly variables
    #  - NQ:     Number of quarterly variables
    #  - i_idio: Logical. Gives index for monthly variables (1) and quarterly (0)
    #
    #Output:
    #  - A:   Transition matrix
    #  - C:   Observation matrix
    #  - Q:   Covariance for transition equation residuals
    #  - R:   Covariance for observation equation residuals
    #  - Z_0: Initial value of state
    #  - V_0: Initial value of covariance matrix

    pC  = Rcon.shape[1]   # Gives 'tent' structure size (quarterly to monthly)
    ppC = max(p,pC)
    n_b = blocks.shape[1] # Number of blocks

    xBal,indNaN = remNaNs_spline(x.copy(),optNaN)  # Spline without NaNs
    
    T,N = xBal.shape  # Time T series number N
    nM  = N-nQ        # Number of monthly series
    
    xNaN         = xBal.copy()
    xNaN[indNaN] = np.nan        # Set missing values equal to NaNs
    res          = xBal.copy()   # Spline output equal to res Later this is used for residuals
    resNaN       = xNaN.copy()   # Later used for residuals

    # Initialize model coefficient output
    C   = None
    A   = None
    Q   = None
    V_0 = None

    # Set the first observations as NaNs: For quarterly-monthly aggreg. scheme
    indNaN[:pC-1, :] = np.True_

    # Set the first observations as NaNs: For quarterly-monthly aggreg. scheme
    for i in range(n_b): # Loop for each block
        r_i = r[0,i].copy() # r_i = 1 when block is loaded

        # Observation equation -----------------------------------------------

        C_i = np.zeros((N, r_i * ppC))     # Initialize state variable matrix helper
        idx_i = np.where(blocks[:, i])[0]  # Returns series index loading block i
        idx_iM = idx_i[idx_i < nM]         # Monthly series indicies for loaded blocks
        idx_iQ = idx_i[idx_i >= nM]        # Quarterly series indicies for loaded blocks

        # Returns eigenvector v w/largest eigenvalue d, CHECK: test if eig values are the same in Matlab
        d, v = eig(np.cov(res[:, idx_iM], rowvar=False))
        e_idx = np.where(d == np.max(d))[0]
        d = d[e_idx]
        v = v[:, e_idx]

        # Flip sign for cleaner output. Gives equivalent results without this section
        if np.sum(v) < 0:
            v = -v

        # For monthly series with loaded blocks (rows), replace with eigenvector
        # This gives the loading
        C_i[idx_iM,0:r_i] = v.copy()
        f                 = np.matmul(res[:,idx_iM],v) # Data projection for eigenvector direction
        F                 = np.array(f[(pC-1):f.shape[0],:]).reshape((-1,1))

        # Lag matrix using loading. This is later used for quarterly series
        for kk in range(1,max(p+1,pC)):
            F = np.concatenate((F,f[(pC-1)-kk:f.shape[0]-kk,:]), axis =1)

        Rcon_i = np.kron(Rcon,np.eye(r_i)) # Quarterly-monthly aggregation scheme
        q_i    = np.kron(q,np.zeros((r_i,1)))

        # Produces projected data with lag structure (so pC-1 fewer entries)
        ff = F[:, 0:(r_i*pC)].copy()

        for j in idx_iQ: # Loop for quarterly variables

            # For series j, values are dropped to accommodate lag structure
            xx_j = resNaN[(pC-1):,j].copy()

            if sum(~np.isnan(xx_j)) < (ff.shape[1] + 2):
                xx_j = res[(pC-1):,j].copy()

            ff_j = ff[~np.isnan(xx_j),:].copy()
            xx_j = xx_j[~np.isnan(xx_j)].reshape((-1,1)).copy()

            iff_j = np.linalg.inv(np.matmul(ff_j.T,ff_j))
            Cc    = np.matmul(np.matmul(iff_j,ff_j.T),xx_j)

            a1 = np.matmul(iff_j,Rcon_i.T)
            a2 = np.linalg.inv(np.matmul(np.matmul(Rcon_i,iff_j),Rcon_i.T))
            a3 = np.matmul(Rcon_i,Cc)-q_i

            # Spline data monthly to quarterly conversion
            Cc = Cc - np.matmul(np.matmul(a1,a2),a3)

            C_i[j,0:pC*r_i] = Cc.T.copy() # Place in output matrix

        # Zeros in first pC-1 entries (replace dropped from lag)
        ff = np.concatenate([np.zeros((pC-1,pC*r_i)),ff], axis = 0)

        # Residual Calculations
        res            = res - np.matmul(ff,C_i.T)
        resNaN         = res.copy()
        resNaN[indNaN] = np.nan

        # Combine past loadings together
        if i == 0:
            C = C_i.copy()
        else:
            C = np.hstack([C,C_i.copy()])

        # Transition equation ------------------------------------------------

        z = F[:,r_i-1].copy()           # Projected data (no lag)
        Z = F[:,r_i:(r_i*(p+1))].copy() # Data with lag 1

        A_i    = np.zeros((r_i*ppC,r_i*ppC)).T                               # Initialize transition matrix
        A_temp = np.matmul(np.matmul(np.linalg.inv(np.matmul(Z.T,Z)),Z.T),z) # OLS: gives coefficient value AR(p) process

        A_i[:r_i,:r_i*p]       = A_temp.T.copy()
        A_i[r_i:,:r_i*(ppC-1)] = np.eye(r_i*(ppC-1))


        Q_i            = np.zeros((ppC*r_i,ppC*r_i))
        e              = z - np.matmul(Z,A_temp) # VAR residuals
        Q_i[:r_i,:r_i] = np.cov(e, rowvar=False) # VAR covariance matrix

        initV_i = np.reshape(np.matmul(np.linalg.inv(np.eye((r_i*ppC)**2) - np.kron(A_i,A_i)),Q_i.flatten('F').reshape((-1,1))),(r_i*ppC,r_i*ppC))

        # Gives top left block for the transition matrix
        if i == 0:
            A   = A_i.copy()
            Q   = Q_i.copy()
            V_0 = initV_i.copy()
        else:
            A   = block_diag(A,A_i)
            Q   = block_diag(Q,Q_i)
            V_0 = block_diag(V_0,initV_i)

    eyeN = np.eye(N)[:,i_idio.flatten('F')] # Used inside observation matrix

    C    = np.hstack([C,eyeN])
    # Monthly-quarterly agreggation scheme
    C    = np.hstack([C,np.vstack([np.zeros((nM,5*nQ)),np.kron(np.eye(nQ),np.array([1,2,3,2,1]).reshape((1,-1)))])])
    # Initialize covariance matrix for transition matrix
    R    = np.diag(np.nanvar(resNaN,ddof = 1,axis = 0))

    ii_idio = np.where(i_idio)[0]        # Indicies for monthly variables
    n_idio  = ii_idio.shape[0]           # Number of monthly variables
    BM      = np.zeros((n_idio,n_idio))  # Initialize monthly transition matrix values
    SM      = np.zeros((n_idio, n_idio)) # Initialize monthly residual covariance matrix values

    for i in range(n_idio): # Loop for monthly variables

        # Set observation equation residual covariance matrix diagonal
        R[ii_idio[i],ii_idio[i]] = 1e-4

        # Subsetting series residuals for series i
        res_i = resNaN[:,ii_idio[i]].copy()

        # Returns number of leading/ending zeros
        try:
            leadZero = np.max(np.where(np.arange(1,T+1) == np.cumsum(np.isnan(res_i)))) + 1
        except ValueError:
            leadZero = None

        try:
            endZero  = -(np.max(np.where(np.arange(1,T+1) == np.cumsum(np.isnan(res_i[::-1])))[0]) + 1)
        except ValueError:
            endZero = None

        # Truncate leading and ending zeros
        res_i = res[:,ii_idio[i]].copy()
        res_i = res_i[:endZero]
        res_i = res_i[leadZero:].reshape((-1,1),order="F")

        # Linear regression: AR 1 process for monthly series residuals
        BM[i,i] = np.matmul(np.matmul(np.linalg.inv(np.matmul(res_i[:-1].T,res_i[:-1])),res_i[:-1].T),res_i[1:])
        SM[i,i] = np.cov(res_i[1:] - (res_i[:-1]*BM[i,i]),rowvar=False)

    Rdiag       = np.diag(R).copy()
    sig_e       = (Rdiag[nM:]/19)
    Rdiag[nM:]  = 1e-4
    R           = np.diag(Rdiag).copy() # Covariance for obs matrix residuals

    # For BQ, SQ
    rho0      = np.array([[.1]])
    temp      = np.zeros((5,5))
    temp[0,0] = 1

    # Blocks for covariance matrices
    SQ = np.kron(np.diag((1 - rho0[0,0]**2)*sig_e),temp)
    BQ = np.kron(np.eye(nQ),np.vstack([np.hstack([rho0,np.zeros((1,4))]),np.hstack([np.eye(4),np.zeros((4,1))])]))

    initViQ = np.matmul(np.linalg.inv(np.eye((5*nQ)**2) - np.kron(BQ,BQ)),SQ.reshape((-1,1))).reshape((5*nQ,5*nQ))
    initViM = np.diag(1/np.diag(np.eye(BM.shape[0]) - BM**2))*SM

    # Output
    A   = block_diag(A,BM,BQ)
    Q   = block_diag(Q,SM,SQ)
    Z_0 = np.zeros((A.shape[0],1))
    V_0 = block_diag(V_0,initViM,initViQ)

    return A, C, Q, R, Z_0, V_0

def EMstep(y, A, C, Q, R, Z_0, V_0, r,p,R_mat,q,nQ,i_idio,blocks):
    #EMstep    Applies EM algorithm for parameter reestimation
    #
    #  Syntax:
    #    [C_new, R_new, A_new, Q_new, Z_0, V_0, loglik]
    #    = EMstep(y, A, C, Q, R, Z_0, V_0, r, p, R_mat, q, nQ, i_idio, blocks)
    #
    #  Description:
    #    EMstep reestimates parameters based on the Estimation Maximization (EM)
    #    algorithm. This is a two-step procedure:
    #    (1) E-step: the expectation of the log-likelihood is calculated using
    #        previous parameter estimates.
    #    (2) M-step: Parameters are re-estimated through the maximisation of
    #        the log-likelihood (maximize result from (1)).
    #
    #    See "Maximum likelihood estimation of factor models on data sets with
    #    arbitrary pattern of missing data" for details about parameter
    #    derivation (Banbura & Modugno, 2010). This procedure is in much the
    #    same spirit.
    #
    #  Input:
    #    y:      Series data
    #    A:      Transition matrix
    #    C:      Observation matrix
    #    Q:      Covariance for transition equation residuals
    #    R:      Covariance for observation matrix residuals
    #    Z_0:    Initial values of factors
    #    V_0:    Initial value of factor covariance matrix
    #    r:      Number of common factors for each block (e.g. vector [1 1 1 1])
    #    p:      Number of lags in transition equation
    #    R_mat:  Estimation structure for quarterly variables (i.e. "tent")
    #    q:      Constraints on loadings
    #    nQ:     Number of quarterly series
    #    i_idio: Indices for monthly variables
    #    blocks: Block structure for each series (i.e. for a series, the structure
    #            [1 0 0 1] indicates loadings on the first and fourth factors)
    #
    #  Output:
    #    C_new: Updated observation matrix
    #    R_new: Updated covariance matrix for residuals of observation matrix
    #    A_new: Updated transition matrix
    #    Q_new: Updated covariance matrix for residuals for transition matrix
    #    Z_0:   Initial value of state
    #    V_0:   Initial value of covariance matrix
    #    loglik: Log likelihood
    #
    # References:
    #   "Maximum likelihood estimation of factor models on data sets with
    #   arbitrary pattern of missing data" by Banbura & Modugno (2010).
    #   Abbreviated as BM2010
    #
    #

    # Initialize preliminary values

    # Store series/model values
    n,T        = y.shape
    nM         = n - nQ
    pC         = R_mat.shape[1]
    ppC        = max(p,pC)
    num_blocks = blocks.shape[1]

    # ESTIMATION STEP: Compute the (expected) sufficient statistics for a single
    # Kalman filter sequence

    # Running the Kalman filter and smoother with current parameters
    # Note that log-liklihood is NOT re-estimated after the runKF step: This
    # effectively gives the previous iteration's log-likelihood
    # For more information on output, see runKF
    Zsmooth,Vsmooth,VVsmooth,loglik = runKF(y, A, C, Q, R, Z_0, V_0)

    # MAXIMIZATION STEP (TRANSITION EQUATION)
    # See (Banbura & Modugno, 2010) for details.

    # Initialize output
    A_new   = A.copy()
    Q_new   = Q.copy()
    V_0_new = V_0.copy()

    # 2A. UPDATE FACTOR PARAMETERS INDIVIDUALLY ----------------------------
    for i in range(num_blocks): # Loop for each block: factors are uncorrelated

        # SETUP INDEXING
        r_i      = r[0,i].copy()         # r_i = 1 if block is loaded
        rp       = r_i*p
        rp1      = np.sum(r[0,:i]) *ppC
        b_subset = np.arange(rp1,rp1+rp) # Subset blocks: Helps for subsetting Zsmooth, Vsmooth
        t_start  = rp1                   # Transition matrix factor idx start
        t_end    = (rp1+r_i*ppC)         # Transition matrix factor idx end

        # ESTIMATE FACTOR PORTION OF Q, A
        # Note: EZZ, EZZ_BB, EZZ_FB are parts of equations 6 and 8 in BM 2010

        # E[f_t*f_t' | Omega_T]
        EZZ    = np.matmul(Zsmooth[b_subset,1:],Zsmooth[b_subset,1:].T) + \
                   np.sum(Vsmooth[1:,[b_subset],[b_subset]], axis =0)

        # E[f_{t-1}*f_{t-1}' | Omega_T]
        EZZ_BB = np.matmul(Zsmooth[b_subset,:-1],Zsmooth[b_subset,:-1].T) + \
                   np.sum(Vsmooth[:-1,[b_subset],[b_subset]],axis =0)

        # E[f_t*f_{t-1}' | Omega_T]
        EZZ_FB = np.matmul(Zsmooth[b_subset,1:],Zsmooth[b_subset,:-1].T) + \
                    np.sum(VVsmooth[:,b_subset,b_subset], axis = 0)

        # Select transition matrix/covariance matrix for block i
        A_i = A[t_start:t_end, t_start:t_end].copy()
        Q_i = Q[t_start:t_end, t_start:t_end].copy()

        # Equation 6: Estimate VAR(p) for factor
        A_i[:r_i,:rp] = np.matmul(EZZ_FB[:r_i,:rp],np.linalg.inv(EZZ_BB[:rp,:rp]))

        # Equation 8: Covariance matrix of residuals of VA
        Q_i[:r_i,:r_i] = (EZZ[:r_i,:r_i] - np.matmul(A_i[:r_i,:rp],EZZ_FB[:r_i,:rp].T))/T

        # Place updated results in output matrix
        A_new[t_start:t_end, t_start:t_end]   = A_i.copy()
        Q_new[t_start:t_end, t_start:t_end]   = Q_i.copy()
        V_0_new[t_start:t_end, t_start:t_end] = Vsmooth[0,t_start:t_end,t_start:t_end].copy()

    # B. UPDATING PARAMETERS FOR IDIOSYNCRATIC COMPONENT ------------------

    rp1      = np.sum(r)*ppC              # Col size of factor portion
    niM      = np.sum(i_idio[:nM])        # Number of monthly values
    t_start  = rp1                        # Start of idiosyncratic component index
    i_subset = np.arange(t_start,rp1+niM) # Gives indices for monthly idiosyncratic component values

    # Below 3 estimate the idiosyncratic component (for eqns 6, 8 BM 2010)

    # E[f_t*f_t' | \Omega_T]
    EZZ = np.diag(np.diag(np.matmul(Zsmooth[t_start:,1:],Zsmooth[t_start:,1:].T))) + \
            np.diag(np.diag(np.sum(Vsmooth[1:,t_start:,t_start:], axis = 0)))

    # E[f_{t-1}*f_{t-1}' | \Omega_T]
    EZZ_BB = np.diag(np.diag(np.matmul(Zsmooth[t_start:,:-1],Zsmooth[t_start:,:-1].T))) + \
                np.diag(np.diag(np.sum(Vsmooth[:-1,t_start:,t_start:], axis =0)))

    # E[f_t*f_{t-1}' | \Omega_T]
    EZZ_FB = np.diag(np.diag(np.matmul(Zsmooth[t_start:,1:], Zsmooth[t_start:,:-1].T))) + \
                np.diag(np.diag(np.sum(VVsmooth[:,t_start:,t_start:], axis = 0)))

    A_i = np.matmul(EZZ_FB,np.diag(1/np.diag((EZZ_BB)))) # Equation 6
    Q_i = (EZZ - np.matmul(A_i,EZZ_FB.T))/T              # Equation 8

    # Place updated results in output matrix
    A_new[np.ix_(i_subset,i_subset)]   = A_i[:niM,:niM].copy()
    Q_new[np.ix_(i_subset,i_subset)]   = Q_i[:niM,:niM].copy()
    V_0_new[np.ix_(i_subset,i_subset)] = np.diag(np.diag(Vsmooth[0,i_subset,i_subset].copy()))

    # 3 MAXIMIZATION STEP (observation equation)

    # INITIALIZATION AND SETUP ----------------------------------------------

    Z_0 = Zsmooth[:,[0]].copy()

    # Set missing data series values to 0
    y              = y.copy()
    nanY           = np.isnan(y).astype(np.int64)
    y[np.isnan(y)] = 0

    # LOADINGS
    C_new = C.copy()

    # Blocks
    bl   = np.unique(blocks,axis =0) # Gives unique loadings
    n_bl = bl.shape[0]               # Number of unique loadings

    for i in range(num_blocks): # Loop through each block
        if i == 0:
            # Initialize indices
            bl_idxQ = np.tile(bl[:,[i]],(1,r[0,i]*ppC))
            bl_idxM = np.hstack([np.tile(bl[:,[i]],(1,r[0,i])),np.zeros((n_bl,r[0,i]*(ppC-1)))])
            R_con   = np.kron(R_mat,np.eye(r[0,i]))
            q_con   = np.zeros((r[0,i]*R_mat.shape[0],1))
        else:
            # Indicator for monthly factor loadings
            bl_idxQ = np.hstack([bl_idxQ, np.tile(bl[:,[i]],(1,r[0,i]*ppC))])

            # Indicator for quarterly factor loadings
            bl_idxM = np.hstack([np.hstack([bl_idxM, np.tile(bl[:,[i]],(1,r[0,i]))]),np.zeros((n_bl,r[0,i]*(ppC-1)))])

            # Block diagonal matrix giving monthly-quarterly aggreg scheme
            R_con   = block_diag(R_con,np.kron(R_mat,np.eye(r[0,i])))
            q_con   = np.vstack([q_con,np.zeros((r[0,i]*R_mat.shape[0],1))])

    #  Indicator for monthly/quarterly blocks in observation matrix
    bl_idxM = bl_idxM == 1
    bl_idxQ = bl_idxQ == 1

    i_idio_M = i_idio[:nM].copy()             # Gives 1 for monthly series
    n_idio_M = np.where(i_idio_M)[0].shape[0] # Number of monthly series
    c_i_idio = np.cumsum(i_idio)              # Cumulative number of monthly series

    for i in range(n_bl): # Loop through unique loadings (e.g. [1 0 0 0], [1 1 0 0])

        bl_i   = bl[[i],:].copy()
        rs     = np.sum(r[np.where(bl_i == 1)])             # Total num of blocks loaded
        idx_i  = np.where((blocks == bl_i).all(axis =1))[0] # Indices for bl_i
        idx_iM = idx_i[idx_i < nM]                          # Only monthly
        n_i    = len(idx_iM)                                # Number of monthly series

        # Initialize sums in equation 13 of BGR 2010
        denom = np.zeros((n_i*rs,n_i*rs))
        nom   = np.zeros((n_i,rs))

        # Stores monthly indicies. These are done for input robustness
        i_idio_i  = i_idio_M[idx_iM,:].flatten('F').copy()
        i_idio_ii = c_i_idio[idx_iM].copy()
        i_idio_ii = i_idio_ii[i_idio_i].copy() - 1

        # UPDATE MONTHLY VARIABLES: Loop through each period ----------------

        # bl_idxM_ind is the same as bl_idxM(i, :) in Matlab
        # It can get a bit messy with long indexing
        bl_idxM_ind = np.where(bl_idxM[i, :])[0]

        for t in range(T):

            # Gives selection matrix (1 for nonmissing values)
            Wt          = np.diag(np.logical_not(nanY[idx_iM,t]).astype(np.int64))

            # E[f_t*t_t' | Omega_T]
            denom += np.kron(np.matmul(Zsmooth[bl_idxM_ind][:,[t+1]], Zsmooth[bl_idxM_ind][:,[t+1]].T) + \
                                        Vsmooth[t+1][np.ix_(bl_idxM_ind,bl_idxM_ind)],
                                    Wt)

            # E[y_t*f_t' | \Omega_T]
            nom += np.matmul(y[idx_iM][:,[t]],Zsmooth[bl_idxM_ind][:,[t+1]].T) - \
                   np.matmul(Wt[:,i_idio_i],
                                    np.matmul(Zsmooth[rp1+i_idio_ii][:,[t+1]],Zsmooth[bl_idxM_ind][:,[t+1]].T) + \
                                    Vsmooth[t+1][rp1+i_idio_ii,:][:,bl_idxM_ind])

        # POSSIBLE WEAK POINT FOUND: NEED TO TEST ON INDEXING AS NUMPY DOES NOT MAINTAIN PROPER MATRIX FORM DEPENDING ON HOW ITS INDEXED: CHECK

        # Eqn 13 BGR 2010
        vec_C = np.matmul(np.linalg.inv(denom),nom.flatten('F').reshape((-1,1)))

        # Place updated monthly results in output matrix
        C_new[np.ix_(idx_iM,bl_idxM_ind)] = vec_C.copy().reshape((n_i,rs),order = "F") # CHECK: RESHAPE NEEDS TO BE VERIFIED

        # UPDATE QUARTERLY VARIABLES -----------------------------------------

        idx_iQ = idx_i[idx_i >=nM].copy() # Index for quarterly series
        rps = rs*ppC

        # Monthly-quarterly aggregation scheme
        R_con_i = R_con[:,bl_idxQ[i,:]]
        q_con_i = q_con.copy()

        no_c = np.where(~(R_con_i.any(axis = 1)))[0]
        R_con_i = np.delete(R_con_i,no_c,axis = 0)
        q_con_i = np.delete(q_con_i, no_c, axis=0)

        # Loop through quarterly series in loading. This parallels monthly code
        for j in idx_iQ:

            # Initialization
            denom = np.zeros((rps,rps))
            nom   = np.zeros((1,rps))

            idx_jQ = j - nM # Ordinal position of quarterly variable
            # Loc of factor structure corresponding to quarterly var residuals
            i_idio_jQ = np.arange(rp1 + n_idio_M + 5*(idx_jQ),rp1 + n_idio_M + 5*(idx_jQ+1))

            # Place quarterly values in output matrix
            V_0_new[np.ix_(i_idio_jQ,i_idio_jQ)] = Vsmooth[0][np.ix_(i_idio_jQ,i_idio_jQ)].copy()
            A_new[i_idio_jQ[0],i_idio_jQ[0]]     = A_i[i_idio_jQ[0]-rp1,i_idio_jQ[0]-rp1].copy()
            Q_new[i_idio_jQ[0],i_idio_jQ[0]]     = Q_i[i_idio_jQ[0]-rp1,i_idio_jQ[0]-rp1].copy()

            # bl_idxQ_ind is the same as bl_idxQ(i,:) in Matlab
            # It can get a bit messy with long indexing
            bl_idxQ_ind = np.where(bl_idxQ[i,:])[0]
            
            for t in range(T):

                # Selection matrix for quarterly values
                Wt = np.diag(np.logical_not(nanY[[j]][:,[t]]).astype(np.int64))

                # Intermediate steps in BGR equation 13
                denom += np.kron(np.matmul(Zsmooth[bl_idxQ_ind][:,[t+1]],Zsmooth[bl_idxQ_ind][:,[t+1]].T) + \
                                 Vsmooth[t+1][np.ix_(bl_idxQ_ind,bl_idxQ_ind)],
                                 Wt)
                nom += y[j,t]*Zsmooth[bl_idxQ_ind,t+1].T
                nom -= np.matmul(Wt,np.matmul(np.matmul(np.array([[1,2,3,2,1]]), Zsmooth[i_idio_jQ][:,[t+1]]),
                                              Zsmooth[bl_idxQ_ind][:,[t+1]].T) + \
                                    np.matmul(np.array([[1,2,3,2,1]]),Vsmooth[t+1][np.ix_(i_idio_jQ,bl_idxQ_ind)]))

            C_i = np.matmul(np.linalg.inv(denom),nom.T)

            # BGR equation 13
            C_i_constr = C_i - np.matmul(np.matmul(np.matmul(np.linalg.inv(denom),R_con_i.T),
                                                   np.linalg.inv(np.matmul(np.matmul(R_con_i,np.linalg.inv(denom)),R_con_i.T))),
                                         np.matmul(R_con_i,C_i)-q_con_i)

            # Place updated values in output structure
            C_new[j,bl_idxQ_ind] = C_i_constr.flatten('F')

    # 3B. UPDATE COVARIANCE OF RESIDUALS FOR OBSERVATION EQUATION -----------
    # Initialize covariance of residuals of observation equation
    R_new = np.zeros((n,n))
    for t in range(T):

        # Selection matrix
        Wt = np.diag(np.logical_not(nanY[:,t])).astype(np.int64)

        # BGR equation 15
        R_new += np.matmul(y[:,[t]] - np.matmul(np.matmul(Wt,C_new),Zsmooth[:,[t+1]]),
                           (y[:,[t]] - np.matmul(np.matmul(Wt,C_new),Zsmooth[:,[t+1]])).T) + \
                 np.matmul(np.matmul(np.matmul(np.matmul(Wt,C_new),Vsmooth[t+1][:,:]),C_new.T),Wt) + \
                 np.matmul(np.matmul((np.eye(n) - Wt),R),(np.eye(n)-Wt))

    i_idio_M = np.where(i_idio_M.flatten('F'))[0]
    R_new        = R_new/T
    RR           = np.diag(R_new).copy() # RR(RR<1e-2) = 1e-2
    RR[i_idio_M] = 1e-4                  # Ensure non-zero measurement error. See Doz, Giannone, Reichlin (2012) for reference.
    RR[nM:]      = 1e-4
    R_new        = np.diag(RR).copy()
    # CHECK: np.diag to ensure no read only and
    return C_new, R_new, A_new, Q_new, Z_0, V_0, loglik

def em_converged(loglik, previous_loglik, threshold = 1e-4, check_decreased = 1):
    
    # em_converged    checks whether EM algorithm has converged
    # 
    #   Syntax:
    #     [converged, decrease] = em_converged(loglik, previous_loglik, threshold, check_increased)
    # 
    #   Description:
    #     em_converged() checks whether EM has converged. Convergence occurs if
    #     the slope of the log-likelihood function falls below 'threshold'(i.e.
    #     f(t) - f(t-1)| / avg < threshold) where avg = (|f(t)| + |f(t-1)|)/2
    #     and f(t) is log lik at iteration t. 'threshold' defaults to 1e-4.
    # 
    #     This stopping criterion is from Numerical Recipes in C (pg. 423).
    #     With MAP estimation (using priors), the likelihood can decrease
    #     even if the mode of the posterior increases.
    # 
    #   Input arguments:
    #     loglik: Log-likelihood from current EM iteration
    #     previous_loglik: Log-likelihood from previous EM iteration
    #     threshold: Convergence threshhold. The default is 1e-4.
    #     check_decreased: Returns text output if log-likelihood decreases.
    # 
    #   Output:
    #     converged (numeric): Returns 1 if convergence criteria satisfied, and 0 otherwise.
    #     decrease (numeric): Returns 1 if loglikelihood decreased.

    # Initialize output
    converged = 0
    decrease  = 0

    # Check if log-likelihood decreases (optional)
    if check_decreased == 1:
        if (loglik - previous_loglik) < -1e-3:
            print('******likelihood decreased from {} to {}').format(previous_loglik,loglik)
            decrease = 1

    # Check convergence criteria
    delta_loglik = np.abs(loglik - previous_loglik) # Difference in loglik
    avg_loglik   = (np.abs(loglik) + np.abs(previous_loglik) + np.finfo(float).eps)/2

    if (delta_loglik/avg_loglik) < threshold:
        converged = 1 # Check convergence

    return converged, decrease

def runKF(Y,A,C,Q,R,Z_0,V_0):
    #runKF()    Applies Kalman filter and fixed-interval smoother
    #
    #  Syntax:
    #    [zsmooth, Vsmooth, VVsmooth, loglik] = runKF(Y, A, C, Q, R, Z_0, V_0)
    #
    #  Description:
    #    runKF() applies a Kalman filter and fixed-interval smoother. The
    #    script uses the following model:
    #           Y_t = C_t Z_t + e_t for e_t ~ N(0, R)
    #           Z_t = A Z_{t-1} + mu_t for mu_t ~ N(0, Q)

    #  Throughout this file:
    #    'm' denotes the number of elements in the state vector Z_t.
    #    'k' denotes the number of elements (observed variables) in Y_t.
    #    'nobs' denotes the number of time periods for which data are observed.
    #
    #  Input parameters:
    #    Y: k-by-nobs matrix of input data
    #    A: m-by-m transition matrix
    #    C: k-by-m observation matrix
    #    Q: m-by-m covariance matrix for transition equation residuals (mu_t)
    #    R: k-by-k covariance for observation matrix residuals (e_t)
    #    Z_0: 1-by-m vector, initial value of state
    #    V_0: m-by-m matrix, initial value of state covariance matrix
    #
    #  Output parameters:
    #    zsmooth: k-by-(nobs+1) matrix, smoothed factor estimates
    #             (i.e. zsmooth(:,t+1) = Z_t|T)
    #    Vsmooth: k-by-k-by-(nobs+1) array, smoothed factor covariance matrices
    #             (i.e. Vsmooth(:,:,t+1) = Cov(Z_t|T))
    #    VVsmooth: k-by-k-by-nobs array, lag 1 factor covariance matrices
    #              (i.e. Cov(Z_t,Z_t-1|T))
    #    loglik: scalar, log-likelihood
    #
    #  References:
    #  - QuantEcon's "A First Look at the Kalman Filter"
    #  - Adapted from replication files for:
    #    "Nowcasting", 2010, (by Marta Banbura, Domenico Giannone and Lucrezia
    #    Reichlin), in Michael P. Clements and David F. Hendry, editors, Oxford
    #    Handbook on Economic Forecasting.
    #
    # The software can be freely used in applications.
    # Users are kindly requested to add acknowledgements to published work and
    # to cite the above reference in any resulting publications

    S = SKF(Y, A, C, Q, R, Z_0, V_0)  # Kalman filter
    S = FIS(A, S)                     # Fixed interval smoother

    # Organize output
    zsmooth  = S["ZmT"].copy()
    Vsmooth  = S["VmT"].copy()
    VVsmooth = S["VmT_1"].copy()
    loglik   = S["loglik"].copy()

    return zsmooth,Vsmooth,VVsmooth,loglik

def SKF(Y, A, C, Q, R, Z_0, V_0):
    # SKF    Applies Kalman filter
    #
    #  Syntax:
    #    S = SKF(Y, A, C, Q, R, Z_0, V_0)
    #
    #  Description:
    #    SKF() applies the Kalman filter

    #  Input parameters:
    #    Y: k-by-nobs matrix of input data
    #    A: m-by-m transition matrix
    #    C: k-by-m observation matrix
    #    Q: m-by-m covariance matrix for transition equation residuals (mu_t)
    #    R: k-by-k covariance for observation matrix residuals (e_t)
    #    Z_0: 1-by-m vector, initial value of state
    #    V_0: m-by-m matrix, initial value of state covariance matrix
    #
    #  Output parameters:
    #    S.Zm: m-by-nobs matrix, prior/predicted factor state vector
    #          (S.Zm(:,t) = Z_t|t-1)
    #    S.ZmU: m-by-(nobs+1) matrix, posterior/updated state vector
    #           (S.Zm(t+1) = Z_t|t)
    #    S.Vm: m-by-m-by-nobs array, prior/predicted covariance of factor
    #          state vector (S.Vm(:,:,t) = V_t|t-1)
    #    S.VmU: m-by-m-by-(nobs+1) array, posterior/updated covariance of
    #           factor state vector (S.VmU(:,:,t+1) = V_t|t)
    #    S.loglik: scalar, value of likelihood function
    #    S.k_t: k-by-m Kalman gain

    # INITIALIZE OUTPUT VALUES ---------------------------------------------
    # Output structure & dimensions of state space matrix
    m    = C.shape[1]

    # Outputs time for data matrix. "number of observations"
    nobs = Y.shape[1]

    # Instantiate output
    S           = {}
    S["Zm"]     = np.zeros((m,nobs))       # Z_t | t-1 (prior)
    S["Vm"]     = np.zeros((nobs,m,m))     # V_t | t-1 (prior)
    S["ZmU"]    = np.zeros((m,nobs + 1))   # Z_t | t (posterior/updated)
    S["VmU"]    = np.zeros((nobs + 1,m,m)) # V_t | t (posterior/updated)
    S["loglik"] = 0

    # SET INITIAL VALUES ----------------------------------------------------

    S["Zm"][:]  = np.nan
    S["Vm"][:]  = np.nan
    S["ZmU"][:] = np.nan
    S["VmU"][:] = np.nan

    Zu = Z_0.copy() # Z_0|0 (In below loop, Zu gives Z_t | t)
    Vu = V_0.copy() # V_0|0 (In below loop, Vu guvse V_t | t)

    # Store initial values
    S["ZmU"][:,[0]] = Zu.copy()
    S["VmU"][0,:,:] = Vu.copy()

    # KALMAN FILTER PROCEDURE ----------------------------------------------
    for t in range(nobs):
        # CALCULATING PRIOR DISTIBUTION----------------------------------

        # Use transition eqn to create prior estimate for factor
        # i.e. Z = Z_t|t-1
        Z = np.matmul(A,Zu)

        # Prior covariance matrix of Z (i.e. V = V_t|t-1)
        # Var(Z) = Var(A*Z + u_t) = Var(A*Z) + Var(\epsilon) =
        # A*Vu*A' + Q
        V = np.matmul(np.matmul(A,Vu),A.T) + Q
        V = .5 * (V + V.T) # Trick to make symmetric

        # CALCULATING POSTERIOR DISTRIBUTION ----------------------------

        # Removes missing series: These are removed from Y, C, and R
        Y_t,C_t,R_t,_ = MissData(Y[:,[t]],C,R)

        # Check if y_t contains no data. If so, replace Zu and Vu with prior.
        if Y_t.shape[0] == 0:
            Zu = Z.copy()
            Vu = V.copy()
        else:
            # Steps for variance and population regression coefficients:
            # Var(c_t*Z_t + e_t) = c_t Var(A) c_t' + Var(u) = c_t*V *c_t' + R
            VC = np.matmul(V,C_t.T)
            iF = np.linalg.inv(np.matmul(C_t,VC) + R_t)

            # Matrix of population regression coefficients (QuantEcon eqn #4)
            VCF = np.matmul(VC,iF)

            # Gives difference between actual and predicted observation
            # matrix values
            innov = Y_t - np.matmul(C_t,Z)

            # Update estimate of factor values (posterior)
            Zu = Z + np.matmul(VCF,innov)

            # Update covariance matrix (posterior) for time t
            Vu = V - np.matmul(VCF,VC.T)
            Vu = .5 * (Vu + Vu.T)

            # Update log likelihood
            S["loglik"] = S["loglik"] + .5*(np.log(np.linalg.det(iF)) - np.matmul(np.matmul(innov.T,iF), innov))[0,0]

        # STORE OUTPUT----------------------------------------------------

        # Store covariance and observation values for t-1 (priors)
        S["Zm"][:,[t]]   = Z.copy()
        S["Vm"][[t],:,:] = V.copy()

        # Store covariance and state values for t (posteriors)
        # i.e. Zu = Z_t|t   & Vu = V_t|t
        S["ZmU"][:,[t+1]]   = Zu.copy()
        S["VmU"][t+1,:,:] = Vu.copy()

    # Store Kalman gain k_t
    if Y_t.shape[0] == 0:
        S["k_t"] = np.zeros((m,m))
    else:
        S["k_t"] = np.matmul(VCF,C_t)

    return S

def FIS(A,S):
    #FIS()    Applies fixed-interval smoother
    #
    #  Syntax:
    #    S = FIS(A, S)
    #
    #  Description:
    #    SKF() applies a fixed-interval smoother, and is used in conjunction
    #    with SKF(). See  page 154 of 'Forecasting, structural time series models
    #    and the Kalman filter' for more details (Harvey, 1990).
    #
    #  Input parameters:
    #    A: m-by-m transition matrix
    #    S: structure returned by SKF()
    #
    #  Output parameters:
    #    S: FIS() adds the following smoothed estimates to the S structure:
    #    - S.ZmT: m-by-(nobs+1) matrix, smoothed states
    #             (S.ZmT(:,t+1) = Z_t|T)
    #    - S.VmT: m-by-m-by-(nobs+1) array, smoothed factor covariance
    #             matrices (S.VmT(:,:,t+1) = V_t|T = Cov(Z_t|T))
    #    - S.VmT_1: m-by-m-by-nobs array, smoothed lag 1 factor covariance
    #               matrices (S.VmT_1(:,:,t) = Cov(Z_t Z_t-1|T))
    #
    #  Model:
    #   Y_t = C_t Z_t + e_t for e_t ~ N(0, R)
    #   Z_t = A Z_{t-1} + mu_t for mu_t ~ N(0, Q)

    # ORGANIZE INPUT ---------------------------------------------------------

    # Initialize output matrices
    m,nobs   = S["Zm"].shape
    S["ZmT"] = np.zeros((m,nobs+1))
    S["VmT"] = np.zeros((nobs+1,m,m))

    # Fill the final period of ZmT, VmT with SKF() posterior values
    S["ZmT"][:,nobs]   = np.squeeze(S["ZmU"][:,nobs])
    S["VmT"][nobs,:,:] = np.squeeze(S["VmU"][nobs,:,:])

    # Initialize VmT_1 lag 1 covariance matrix for final period
    VmT_1_init             = np.matmul(np.matmul(np.eye(m) - S["k_t"],A),np.squeeze(S["VmU"][nobs-1,:,:]))
    S["VmT_1"]             = np.zeros((nobs,VmT_1_init.shape[0],VmT_1_init.shape[1]))
    S["VmT_1"][nobs-1,:,:] = VmT_1_init

    # Used for recursion process. See companion file for details
    J_2 = np.matmul(np.matmul(np.squeeze(S["VmU"][nobs-1,:,:]),A.T),np.linalg.pinv(np.squeeze(S["Vm"][nobs-1,:,:])))

    # RUN SMOOTHING ALGORITHM ----------------------------------------------
    for t in range(nobs)[::-1]: # Loop through time reverse-chronologically (starting at final period nobs)

        # Store posterior and prior factor covariance values
        VmU = np.squeeze(S["VmU"][t,:,:])
        Vm1 = np.squeeze(S["Vm"][t,:,:])

        # Store previous period smoothed factor covariance and lag-1 covariance
        V_T  = np.squeeze(S["VmT"][t+1,:,:])
        V_T1 = np.squeeze(S["VmT_1"][t,:,:])

        J_1 = J_2.copy()

        # Update smoothed factor estimate
        S["ZmT"][:,[t]] = S["ZmU"][:,[t]] + np.matmul(J_1,S["ZmT"][:,[t+1]] - np.matmul(A,S["ZmU"][:,[t]]))

        # Update smoothed factor covariance matrix
        S["VmT"][t,:,:] = VmU + np.matmul(J_1,np.matmul((V_T - Vm1),J_1.T))

        if t>0:
            # Update weight
            J_2  = np.matmul(np.matmul(np.squeeze(S["VmU"][t-1,:,:]),A.T),np.linalg.pinv(np.squeeze(S["Vm"][t-1,:,:])))

            # Update lag 1 factor covariance matrix
            S["VmT_1"][t-1,:,:] = np.matmul(VmU,J_2.T) + np.matmul(J_1,np.matmul(V_T1 - np.matmul(A,VmU),J_2.T))
    return S

def MissData(y,C,R):
    # Syntax:
    # Description:
    #   Eliminates the rows in y & matrices C, R that correspond to missing
    #   data (NaN) in y
    #
    # Input:
    #   y: Vector of observations at time t
    #   C: Observation matrix
    #   R: Covariance for observation matrix residuals
    #
    # Output:
    #   y: Vector of observations at time t (reduced)
    #   C: Observation matrix (reduced)
    #   R: Covariance for observation matrix residuals
    #   L: Used to restore standard dimensions(n x #) where # is the nr of
    #      available data in y

    # Returns 1 for nonmissing series
    ix = np.where(~np.isnan(y).flatten('F'))[0]

    # Index for columns with nonmissing variables
    e  = np.eye(y.shape[0])
    L  = e[:,ix].copy()

    # Removes missing series
    y  = y[ix].copy()

    # Removes missing series from observation matrix
    C  =  C[ix,:].copy()

    # Removes missing series from transition matrix
    R  =  R[np.ix_(ix,ix)].copy()

    return y,C,R,L

In [ ]:
# Summarize (summarize.py)

#-------------------------------------------------Libraries
import pandas as pd
import numpy as np
from datetime import datetime as dt


#-------------------------------------------------Functions
def summarize(X, Time, Spec):
    """
    summarize Display the detail table for data entering the DFM

    Description:
        Display the detail table for the nowcast, decomposing nowcast changes
        into news and impacts for released data series.
    """
    print("\n\n\n")
    print('Table 2: Data Summary \n')

    # Number of rows and data series
    T,N = X.shape

    print("N =     {} data series".format(N))
    print("T =     {} observations from {} to {} \n".format(T,
                                                         dt.fromordinal(Time[0] - 366).strftime('%Y-%m-%d'),
                                                         dt.fromordinal(Time[-1] - 366).strftime('%Y-%m-%d')))

    # create base table to add additional columns
    base = pd.DataFrame(X, columns=Spec.SeriesName)

    # Create additional columns
    time_range    = base.apply(lambda x: data_table_prep(1,x,Time = Time)).values
    max_min_range = base.apply(lambda x: data_table_prep(2,x,Time = Time)).values
    Frequency     = data_table_prep(3,freq_dict={'m':"Monthly", "q":"Quarterly"},Spec=Spec)
    Units         = data_table_prep(4, N=N,Spec=Spec)

    # Transform base dataframe
    base = base.describe().T
    base["Observation Time Range"] = time_range
    base["Min/Max Dates"] = max_min_range
    base["Frequency"] = Frequency
    base["Units"] = Units

    # Rename columns and set the order of columns
    base.rename(columns={"count": "Observations"}, inplace=True)
    orderColumns = ["Observations",
                    "Observation Time Range",
                    "Units",
                    "Frequency",
                    "mean",
                    "std",
                    "min",
                    "Min/Max Dates"]

    # Rename index values of dataframe
    base.rename(index = {base.index[i]:base.index[i] + " [{}]".format(Spec.SeriesID[i])
                         for i in range(N)},
                inplace = True)

    # Print dataframe
    print(base[orderColumns])

def data_table_prep(option,x=None,freq_dict=None,Time=None,N=None,Spec=None):
    """
    The param option takes values from 1 to 4

    Option 1:
        :param x: a pd.Series
        :param Time: an array containing ordinal values

        1.) Find values that are not NA
        2.) Do a cumsum
        3.) Find the first value that is not NA (will be 1)
        4.) Find the max value
        5.) First value and max value are the first and last date
        :return: a string containing the start and last date values that are not NA

    Option 2:
        :param x: a pd.Series
        :param Time: an array containing ordinal values

        1.) Get the index of the min and max values of x
        2.) Get the Time value in respect to the two indices
        :return: a string containing dates that represent min and max values

    Option 3:
        :param freq_dict: dictionary containing abbreviated as keys with values that are full names: e.g. "q" -> "Quarterly"
        1.) Convert the abbreviated form of frequency to full name
         :return: a string containing full name of the frequency

    Option 4:
        :param N: Number of data series
        :param Spec: Spec class object
        :return: returns a string representing unit transformed
    """

    if option == 1:
        truth   = (np.isnan(x) == False).cumsum()
        first   = Time[np.where(truth == 1)[0][0]]
        last    = Time[np.max(truth) - 1] # subtracted by 1 to get index value
        return "{} to {}".format(dt.fromordinal(first - 366).strftime('%b-%Y'),
                                dt.fromordinal(last - 366).strftime('%b-%Y'))
    elif option == 2:
        min = Time[np.where(x == np.min(x))[0][0]]
        max = Time[np.where(x == np.max(x))[0][0]]

        return "{} / {}".format(dt.fromordinal(min - 366).strftime('%b-%Y'),
                                dt.fromordinal(max - 366).strftime('%b-%Y'))

    elif option == 3:
        return [freq_dict[i] for i in Spec.Frequency]

    elif option == 4:
        return [unitTransformed(Spec.Units[i],
                                Spec.Transformation[i],
                                Spec.Frequency[i])
                for i in range(N)]

    else:
        ValueError("Option needed: must be 1 or 2")

def unitTransformed(unit,transform,freq):
    """
    :param unit: a data series' unit type
    :param transform: what transformation was applied to the data series
    :param freq: the frequency of the data series
    :return: returns a string representing unit transformed
    """
    if unit == "Index":
        unit_transformed = "Index"

    elif transform == "chg":
        if "%" in unit:
            unit_transformed = "Ppt. change"
        else:
            unit_transformed = "Level change"

    elif "pch" == transform and freq == "m":
        unit_transformed = "MoM %"
    elif "pca" == transform and freq == "q":
        unit_transformed = "QoQ AR %"
    else:
        unit_transformed = unit + " [{}]".format(transform)

    return unit_transformed

In [ ]:
#-------------------------------------------------Libraries
from datetime import datetime as dt
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
#from Functions.dfm import SKF,FIS


#-------------------------------------------------update_nowcast
def update_nowcast(X_old,X_new,Time,Spec,Res,series,period,vintage_old,vintage_new):

    if not type(vintage_old) == type(0):
        vintage_old = dt.strptime(vintage_old, '%Y-%m-%d').date().toordinal() + 366

    if not type(vintage_new) == type(0):
        vintage_new = dt.strptime(vintage_new, '%Y-%m-%d').date().toordinal() + 366

    # Make sure datasets are the same size
    N     = np.shape(X_new)[1]

    # append 1 year (12 months) of data to each dataset to allow for
    # forecasting at different horizons
    T_old = np.shape(X_old)[0]
    T_new = np.shape(X_new)[0]

    if T_new > T_old:

        temp    = np.zeros((T_new - T_old,N))
        temp[:] = np.nan
        X_old   = np.vstack([X_old,temp])

    temp    = np.zeros((12, N))
    temp[:] = np.nan

    # append 1 year (12 months) of data to each dataset to allow for
    # forecasting at different horizons
    X_old = np.vstack([X_old, temp])
    X_new = np.vstack([X_new, temp])

    future = np.array([(dt.fromordinal(Time[-1] - 366) +
                           relativedelta(months=+ i)).toordinal() + 366 for i in range(1,13)])

    Time = np.hstack([Time,future])

    i_series    = np.where(series == Spec.SeriesID)[0]
    series_name = Spec.SeriesName[i_series]
    freq        = Spec.Frequency[i_series][0]

    if freq == 'm':
        y, m      = period.split(freq)
        y         = int(y)
        m         = int(m)
        d         = 1
        t_nowcast = np.where((dt(y,m,d).toordinal()+366) == Time)[0]

    elif freq == 'q':
        y, q      = period.split(freq)
        y         = int(y)
        m         = 3 * int(q)
        d         = 1
        t_nowcast = np.where((dt(y,m,d).toordinal()+366) == Time)[0]

    else:
        return "Freq's value is not appropiate"

    if t_nowcast.size == 0:
        ValueError('Period is out of nowcasting horizon (up to one year ahead).')

    # Update nowcast for target variable 'series' (i) at horizon 'period' (t)
    #   > Relate nowcast update into news from data releases:
    #     a. Compute the impact from data revisions
    #     b. Compute the impact from new data releases

    X_rev                  = X_new.copy()
    X_rev[np.isnan(X_old)] = np.nan

    # Compute news --------------------------------------------------------

    # Compute impact from data revisions
    y_old,_,_,_,_,_,_,_,_ = News_DFM(X_old, X_rev, Res, t_nowcast, i_series)

    # Display output
    y_rev,y_new,_,actual,forecast,weight,_,_,_ = News_DFM(X_rev,X_new,Res,t_nowcast,i_series)

    print("\n Nowcast Update: {}".format(dt.fromordinal(vintage_new-366).isoformat().split('T')[0]))
    print("\n Nowcast for: {} ({}), {}".format(Spec.SeriesName[i_series][0],
                                               Spec.UnitsTransformed[i_series][0],
                                               pd.to_datetime(dt.fromordinal(Time[t_nowcast]-366)).to_period('Q')))

    # Only display table output if a forecast is made
    if forecast.shape[0] == 0:
        print("\n No forecast was made \n")
    else:
        impact_revisions = y_rev - y_old     # Impact from revisions
        news             = actual - forecast # News from releases
        impact_releases  = weight * news     # Impact of releases

        # Store results
        news_table = pd.DataFrame({'Forecast': forecast.flatten('F'),
                                   'Actual'  : actual.flatten('F'),
                                   'Weight'  : weight.flatten('F'),
                                   'Impact'  : impact_releases.flatten('F')},
                                  index = Spec.SeriesID)

        # Select only series with updates
        data_released = np.any(np.isnan(X_old) & ~np.isnan(X_new),0)

        # Display the impact decomposition
        print('\n Nowcast Impact Decomposition')
        print(' Note: The displayed output is subject to rounding error\n\n')
        print('              {} nowcast:              {:.5f}'.format(dt.fromordinal(vintage_old-366).isoformat().split('T')[0],y_old[0]))
        print('      Impact from data revisions:      {:.5f}'.format(impact_revisions[0]))
        print('       Impact from data releases:      {:.5f}'.format(np.nansum(news_table.Impact)))
        print('                                     +_________')
        print('                    Total impact:      {:.5f}'.format(impact_revisions[0] + np.nansum(news_table.Impact)))
        print('              {} nowcast:              {:.5f}'.format(dt.fromordinal(vintage_new-366).isoformat().split('T')[0],
                                                                                y_new[0]))

        print('\n  Nowcast Detail Table \n')
        print(news_table.iloc[np.where(data_released)[0],:])

def News_DFM(X_old,X_new,Res,t_fcst,v_news):
    # News_DFM()    Calculates changes in news
    # Syntax:
    # [y_old, y_new, singlenews, actual, fore, weight ,t_miss, v_miss, innov] = ...
    #   News_DFM(X_old, X_new, Q, t_fcst, v_news)
    #
    # Description:
    #  News DFM() inputs two datasets, DFM parameters, target time index, and
    #  target variable index. The function then produces Nowcast updates and
    #  decomposes the changes into news.
    #
    # Input Arguments:
    #   X_old:  Old data matrix (old vintage)
    #   X_new:  New data matrix (new vintage)
    #   Res:    DFM() output results (see DFM for more details)
    #   t_fcst: Index for target time
    #   v_news: Index for target variable
    #
    # Output Arguments:
    #   y_old:       Old nowcast
    #   y_new:       New nowcast
    #   single_news: News for each data series
    #   actual:      Observed series release values
    #   fore:        Forecasted series values
    #   weight:      News weight
    #   t_miss:      Time index for data releases
    #   v_miss:      Series index for data releases
    #   innov:       Difference between observed and predicted series values ("innovation")

    # Initialize variables
    r          = Res["C"].shape[1]
    N          = X_new.shape[1]
    singlenews = np.zeros((1,N)) # Initialize news vector (will store news for each series)


    # NO FORECAST CASE: Already values for variables v_news at time t_fcst
    if ~np.isnan(X_new[t_fcst,v_news])[0]:

        Res_old = para_const(X_old,Res,0) # Apply Kalman filter for old data

        y_old   = np.zeros((1,v_news.shape[0]))
        y_new   = np.zeros((1, v_news.shape[0]))
        for i in range(v_news.shape[0]): # Loop for each target variable

            # (Observed value) - (predicted value)
            singlenews[:,v_news[i]] = X_new[t_fcst,v_news[i]] - Res_old["X_sm"][t_fcst,v_news[i]]

            # Set predicted and observed y values
            y_old[0,i] = Res_old["X_sm"][t_fcst,v_news[i]].copy()
            y_new[0,i] = X_new[t_fcst, v_news[i]].copy()

        # Forecast-related output set to empty
        return y_old,y_new,singlenews,None,None,None,None,None,None

    else:
        # FORECAST CASE (these are broken down into (A) and (B))

        # Initialize series mean/standard deviation respectively
        Mx = Res["Mx"].reshape((-1,1))
        Wx = Res["Wx"].reshape((-1,1))

        # Calculate indicators for missing values (1 if missing, 0 otherwise)
        miss_old = np.isnan(X_old).astype(np.int64)
        miss_new = np.isnan(X_new).astype(np.int64)

        # Indicator for missing--combine above information to single matrix where:
        # (i) -1: Value is in the old data, but missing in new data
        # (ii) 1: Value is in the new data, but missing in old data
        # (iii) 0: Values are missing from/available in both datasets
        i_miss = miss_old - miss_new

        # Time/variable indicies where case (b) is true
        t_miss, v_miss = np.where(i_miss == 1)
        ordered_col    = v_miss.argsort()
        t_miss, v_miss = t_miss[ordered_col], v_miss[ordered_col]

        # FORECAST SUBCASE (A): NO NEW INFORMATION
        if v_miss.shape[0] == 0:

            # Fill in missing variables using a Kalman filter
            Res_old = para_const(X_old, Res, 0)
            Res_new = para_const(X_new, Res, 0) # CHECK: Why isn't this being used?

            # Set predicted and observed y values. New y value is set to old
            y_old = Res_old["X_sm"][t_fcst,v_news]
            y_new = y_old.copy()

            # No news, so nothing returned for news-related output
            return y_old,y_new,singlenews,None,None,None,None,None,None

        else:
            #----------------------------------------------------------------------
            #     v_miss=[1:size(X_new,2)]';
            #     t_miss=t_miss(1)*ones(size(X_new,2),1);
            #----------------------------------------------------------------------
            # FORECAST SUBCASE (B): NEW INFORMATION

            # Difference between forecast time and new data time
            lag = t_fcst - t_miss

            # Gives biggest time interval between forecast and new data
            k   = np.max(np.hstack([np.abs(lag),np.max(lag) - np.min(lag)]))

            C = Res["C"].copy() # Observation matrix
            R = Res["R"].copy() # Covariance for observation matrix residuals

            # Number of new events
            n_news = lag.shape[0]

            # Smooth old dataset
            Res_old = para_const(X_old, Res, k)
            Plag    = Res_old["Plag"].copy()

            # Smooth new dataset
            Res_new = para_const(X_new, Res, 0)

            # Subset for target variable and forecast time
            y_old = Res_old["X_sm"][t_fcst,v_news]
            y_new = Res_new["X_sm"][t_fcst,v_news]

            P  = Res_old["P"][1:].copy()

            for i in range(n_news): # Cycle through total number of updates
                h = abs(t_fcst-t_miss[i])[0]
                m = np.maximum(t_miss[i],t_fcst)[0]

                # If location of update is later than the forecasting date
                if t_miss[i] > t_fcst:
                    Pp = Plag[h][m].copy()
                else:
                    Pp = Plag[h][m].T.copy()
                if i == 0:
                    # Initialize projection onto updates
                    P1 = np.matmul(Pp,C[[v_miss[i]]][:,:r].T)
                else:
                    # Projection on updates
                    P1 = np.hstack([P1,np.matmul(Pp,C[[v_miss[i]]][:,:r].T)])

            for i in range(t_miss.shape[0]):
                # Standardize predicted and observed values
                X_new_norm = (X_new[t_miss[i],v_miss[i]] - Mx[v_miss[i]])/Wx[v_miss[i]]
                X_sm_norm  = (Res_old["X_sm"][t_miss[i],v_miss[i]] - Mx[v_miss[i]])/Wx[v_miss[i]]

                # Innovation: Gives [observed] data - [predicted data]
                if i == 0:
                    innov = X_new_norm - X_sm_norm
                else:
                    innov = np.hstack([innov,X_new_norm - X_sm_norm])
            innov = innov.reshape((1,-1))

            ins   = len(innov)
            WW    = np.zeros((v_miss[-1]+1,v_miss[-1]+1))
            WW[:] = np.nan

            # Gives non-standardized series weights
            for i in range(lag.shape[0]):
                for j in range(lag.shape[0]):
                    h = abs(lag[i] - lag[j])
                    m = max(t_miss[i],t_miss[j])

                    if t_miss[j] > t_miss[i]:
                        Pp = Plag[h][m].copy()
                    else:
                        Pp = Plag[h][m].T.copy()

                    if v_miss[i] == v_miss[j] and t_miss[i] != t_miss[j]:
                        WW[v_miss[i],v_miss[j]] = 0
                    else:
                        WW[v_miss[i], v_miss[j]] = R[v_miss[i],v_miss[j]].copy()

                    if j == 0:
                        p2 = np.matmul(np.matmul(C[[v_miss[i]]][:,:r],Pp),C[[v_miss[j]]][:,:r].T) + WW[v_miss[i], v_miss[j]]
                    else:
                        p2 = np.hstack([p2,np.matmul(np.matmul(C[[v_miss[i]]][:,:r],Pp),C[[v_miss[j]]][:,:r].T) + WW[v_miss[i], v_miss[j]]])
                if i == 0:
                    P2 = p2.copy()
                else:
                    P2 = np.vstack([P2,p2])

            try:
                del temp
            except NameError:
                pass

            # CHECK: can this be written better?
            for i in range(v_news.shape[0]): # loop on v_news
                # Convert to real units (unstadardized data)
                if i == 0:
                    totnews = np.matmul(np.matmul(np.matmul(np.matmul(Wx[[v_news[i]]],C[[v_news[i]]][:,:r]),P1),np.linalg.inv(P2)),innov.T)
                    temp    = np.matmul(np.matmul(np.matmul(Wx[[v_news[i]]],C[[v_news[i]]][:,:r]),P1),np.linalg.inv(P2)) * innov
                    gain    = np.matmul(np.matmul(np.matmul(Wx[[v_news[i]]],C[[v_news[i]]][:,:r]),P1),np.linalg.inv(P2))

                    temp = temp.reshape((1,*temp.shape))
                    gain = gain.reshape((1,*gain.shape))
                else:
                    temp_A = np.matmul(np.matmul(np.matmul(np.matmul(Wx[v_news[i]],C[[v_news[i]]][:,:r]),P1),np.linalg.inv(P2)),innov.T)
                    temp_B = np.matmul(np.matmul(np.matmul(Wx[v_news[i]],C[[v_news[i]]][:,:r]),P1),np.linalg.inv(P2)) * innov
                    temp_C = np.matmul(np.matmul(np.matmul(Wx[v_news[i]],C[[v_news[i]]][:,:r]),P1),np.linalg.inv(P2))

                    totnews = np.hstack([totnews, temp_A])
                    temp    = np.vstack([temp,temp_B[np.newaxis,]])
                    gain    = np.vstack([gain, temp_C[np.newaxis,]])

            # Initialize output objects
            singlenews = np.zeros((v_news.shape[0],np.max(t_miss) - np.min(t_miss)+1,N))
            actual     = np.zeros((N,1))
            forecast   = np.zeros((N,1))
            weight     = np.zeros((v_news.shape[0],N,1))
            singlenews[:], actual[:], forecast[:], weight[:] = np.nan,np.nan,np.nan,np.nan

            # Fill in output values
            for i in range(innov.shape[1]):
                actual[v_miss[i],0]   = X_new[t_miss[i],v_miss[i]].copy()
                forecast[v_miss[i],0] = Res_old["X_sm"][t_miss[i],v_miss[i]].copy()

                for j in range(v_news.shape[0]):
                    singlenews[j,t_miss[i]-min(t_miss),v_miss[i]] = temp[j,0,i].copy()
                    weight[j,v_miss[i],:] = gain[j,:,i]/Wx[v_miss[i]]

            singlenews = np.sum(singlenews, axis = 0) # Returns total news
            v_miss     = np.sort(np.unique(v_miss))

    # CHECK: weight seems suspicious
    return y_old,y_new,singlenews,actual,forecast,weight[0],t_miss,v_miss,innov

def para_const(X,P,lag):
    # para_const()    Implements Kalman filter for "News_DFM.m"
    #
    #   Syntax:
    #     Res = para_const(X,P,lag)
    #
    #   Description:
    #     para_const() implements the Kalman filter for the news calculation
    #     step. This procedure smooths and fills in missing data for a given
    #     data matrix X. In contrast to runKF(), this function is used when
    #     model parameters are already estimated.
    #
    #   Input parameters:
    #     X: Data matrix.
    #     P: Parameters from the dynamic factor model.
    #     lag: Number of lags
    #
    #   Output parameters:
    #     Res [struc]: A structure containing the following:
    #       Res.Plag: Smoothed factor covariance for transition matrix
    #       Res.P:    Smoothed factor covariance matrix
    #       Res.X_sm: Smoothed data matrix
    #       Res.F:    Smoothed factors
    #
    #
    #
    #  Kalman filter with specified paramaters
    #  written for
    #  "MAXIMUM LIKELIHOOD ESTIMATION OF FACTOR MODELS ON DATA SETS WITH
    #  ARBITRARY PATTERN OF MISSING DATA."
    #  by Marta Banbura and Michele Modugno
    #
    #   Set model parameters and data preparation

    # Set model parameters
    Z_0 = P["Z_0"].copy()
    V_0 = P["V_0"].copy()
    A   = P["A"].copy()
    C   = P["C"].copy()
    Q   = P["Q"].copy()
    R   = P["R"].copy()
    Mx  = P["Mx"].copy()
    Wx  = P["Wx"].copy()

    # Prepare data
    T = X.shape[0]

    # Standardise x
    Y = ((X - np.tile(Mx,(T,1)))/np.tile(Wx,(T,1))).T

    # Apply Kalman filter and smoother
    # See runKF() for details about FIS and SKF
    Sf = SKF(Y,A,C,Q,R,Z_0,V_0) # Kalman filter
    Ss = FIS(A,Sf)              # Smoothing step

    # Calculate parameter output
    Vs      = Ss["VmT"][1:,:,:].copy() # Smoothed factor covariance for transition matrix
    Vf      = Ss["VmU"][1:,:,:].copy() # Filtered factor posterior covariance
    Zsmooth = Ss["ZmT"].copy()         # Smoothed factors
    Vsmooth = Ss["VmT"].copy()         # Smoothed covariance values

    Plag = [Vs.copy()]


    for jk in range(lag):
        Plag.append(np.zeros(Vs.shape))
        for jt in range(Plag[0].shape[0]-1,lag-1,-1):
            As = np.matmul(np.matmul(Vf[jt-jk-1],A.T),
                           np.linalg.pinv(np.matmul(np.matmul(A,Vf[jt-jk-1]),A.T) + Q))
            Plag[jk+1][jt] = np.matmul(As, Plag[jk][jt])

    # Prepare data for output
    Zsmooth = Zsmooth.T

    x_sm = np.matmul(Zsmooth[1:,:],C.T)                 # Factors to series representation
    X_sm = np.tile(Wx,(T,1)) * x_sm + np.tile(Mx,(T,1)) # Standardized to unstandardized

    # Loading dictionary with the results
    Res = {"Plag" : Plag,
           "P"    : Vsmooth,
           "X_sm" : X_sm,
           "F"    : Zsmooth[1:,:]
    }

    return Res

In [ ]:
#-------------------------------------------------Libraries
import os
from datetime import datetime as dt
#from Functions.load_spec import load_spec
#from Functions.load_data import load_data
#from Functions.dfm import dfm
import pickle
#from Functions.summarize import summarize
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np


#-------------------------------------------------Set dataframe to full view
pd.set_option('display.expand_frame_repr', False)


#-------------------------------------------------User Inputs
vintage      = '2016-06-29'                                                   # vintage dataset to use for estimation
country      = 'US'                                                           # United States macroeconomic data
sample_start = dt.strptime("2000-01-01", '%Y-%m-%d').date().toordinal() + 366 # estimation sample


#-------------------------------------------------Load model specification and dataset.
# Load model specification structure `Spec`
Spec = load_spec('/content/gdrive/MyDrive/Nowcasting-Python-master/Spec_US_example.xls')

# Parse `Spec`
SeriesID         = Spec.SeriesID
SeriesName       = Spec.SeriesName
Units            = Spec.Units
UnitsTransformed = Spec.UnitsTransformed

# Load data
datafile   = os.path.join('/content/gdrive/MyDrive/Nowcasting-Python-master/data',country,vintage + '.xls')
X,Time,Z   = load_data(datafile,Spec,sample_start)

# Summarize dataset
summarize(X,Time,Spec)


#-------------------------------------------------Plot data
# Raw vs transformed
idxSeries = np.where(Spec.SeriesID == "INDPRO")[0][0]
t_obs     = ~np.isnan(X[:,idxSeries])

fig = make_subplots(rows=2, cols=1,
                    subplot_titles=("Raw Observed Data", "Transformed Data"))

fig.append_trace(go.Scatter(
    x=[dt.fromordinal(i - 366).strftime('%Y-%m-%d') for i in Time[t_obs]],
    y=Z[t_obs,idxSeries],
), row=1, col=1)

fig.append_trace(go.Scatter(
    x=[dt.fromordinal(i - 366).strftime('%Y-%m-%d') for i in Time[t_obs]],
    y=X[t_obs,idxSeries],
), row=2, col=1)


fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'} ,
                  title_text="Raw vs Transformed Data",
                  showlegend=False)
fig.update_yaxes(title_text=Spec.Units[idxSeries], row=1, col=1)
fig.update_yaxes(title_text=Spec.UnitsTransformed[idxSeries], row=2, col=1)
fig.show()


#-------------------------------------------------Run dynamic factor model (DFM) and save estimation output as 'ResDFM'.
threshold = 1e-4 # Set to 1e-5 for more robust estimates
Res = dfm(X,Spec,threshold)
Res = {"Res": Res,"Spec":Spec}

with open('ResDFM.pickle', 'wb') as handle:
    pickle.dump(Res, handle)
# TODO: Res and Spec should be separate, this will be fixed after the unit tests are created


#-------------------------------------------------Plot Loglik across number of steps
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(1,len(Res["Res"]["loglik"][1:])+1),
                         y=Res["Res"]["loglik"][1:],
                         mode='lines',
                         name="LogLik")
)
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'} ,
                  title_text="LogLik across number of steps taken",
                  showlegend=False
)
fig.update_yaxes(title_text="LogLik")
fig.update_xaxes(title_text="Number of steps")
fig.show()


#-------------------------------------------------Plot common factor and standardized data.
# select INDPRO data series
idxSeries = np.where(Spec.SeriesID == "INDPRO")[0][0]

# Create traces
fig = go.Figure()
for i in range(Res["Res"]["x_sm"].shape[1]):
    fig.add_trace(go.Scatter(x=[dt.fromordinal(i - 366).strftime('%Y-%m-%d') for i in Time],
                             y=Res["Res"]["x_sm"][:,i],
                             mode='lines',
                             name=Spec.SeriesID[i],
                             line={'width':.9})
)
fig.add_trace(go.Scatter(x=[dt.fromordinal(i - 366).strftime('%Y-%m-%d') for i in Time],
                         y=Res["Res"]["Z"][:,0]*Res["Res"]["C"][idxSeries,0],
                         mode='lines',
                         name="Common Factor",
                         line=dict(color='black', width=1.5))
)

# Plot common factor and standardized data
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'} ,
                  title_text="Common Factor and Standardized Data"
)
fig.show()


#-------------------------------------------------Plot projection of common factor onto Payroll Employment and GDP
# Two plots in one graph
fig = make_subplots(rows=2, cols=1,
                    subplot_titles=("Payroll Employment", "Real Gross Domestic Product"))

# Create an array of the data series that we are interested in looping through to plot the projection
series = ["PAYEMS","GDPC1"]

# For a particular series:
#       1.) plot the common factor
#       2.) plot the data series (with NAs removed)
for i in range(len(series)):

    idxSeries    = np.where(Spec.SeriesID == series[i])[0][0]
    t_obs        = ~np.isnan(X[:,idxSeries])

    CommonFactor = np.matmul(Res["Res"]["C"][idxSeries,:5].reshape(1,-1),Res["Res"]["Z"][:,:5].T) * \
                   Res["Res"]["Wx"][idxSeries] + Res["Res"]["Mx"][idxSeries]

    fig.append_trace(go.Scatter(
        x=[dt.fromordinal(i - 366).strftime('%Y-%m-%d') for i in Time],
        y=CommonFactor[0,:],
        name="Common Factor ({})".format(series[i])
    ), row=i+1, col=1)

    fig.append_trace(go.Scatter(
        x=[dt.fromordinal(i - 366).strftime('%Y-%m-%d') for i in Time[t_obs]],
        y=X[t_obs,idxSeries],
        name="Data ({})".format(series[i])
    ), row=i+1, col=1)

    fig.update_yaxes(title_text=Spec.Units[idxSeries] + " ({})".format(Spec.UnitsTransformed[idxSeries]), row=i+1, col=1)

fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'} ,
                  title_text="Projection of Common Factor")
fig.show()

In [ ]:
#-------------------------------------------------Libraries
#from Functions.load_data import load_data
#from Functions.load_spec import load_spec
#from Functions.update_Nowcast import update_nowcast
import os
import pickle


#-------------------------------------------------User Inputs
series = 'GDPC1'  # Nowcasting real GDP (GDPC1) <fred.stlouisfed.org/series/GDPC1>
period = '2016q4' # Forecasting target quarter


#-------------------------------------------------Load model specification and first vintage of data.
Spec   = load_spec('/content/gdrive/MyDrive/Nowcasting-Python-master/Spec_US_example.xls')


#-------------------------------------------------Load DFM estimation results structure `Res`
with open('ResDFM.pickle', 'rb') as handle:
    Res = pickle.load(handle)


#-------------------------------------------------Update nowcast and decompose nowcast changes into news.
# Nowcast update from week of December 7 to week of December 16, 2016
vintage_old  = '2016-12-16'
vintage_new  = '2016-12-23'
datafile_old = os.path.join('/content/gdrive/MyDrive/Nowcasting-Python-master/data','US',vintage_old + '.xls')
datafile_new = os.path.join('/content/gdrive/MyDrive/Nowcasting-Python-master/data','US',vintage_new + '.xls')

# Load datasets for each vintage
X_old,_,_    = load_data(datafile_old,Spec)
X_new,Time,_ = load_data(datafile_new,Spec)

Res = Res['Res']

update_nowcast(X_old,X_new,Time,Spec,Res,series,period,vintage_old,vintage_new)